In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("pyspark")
sc = SparkContext(conf=conf)

In [2]:
#btc_raw = sc.textFile("graph_datasets/btc_02.txt")
btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6)])

In [24]:
#btc = btc_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], [set(x[1]), True]))
btc = btc_raw.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], [set(x[1]), True]))

In [25]:
btc.collect()

[(0, [{1}, True]),
 (8, [{5, 7}, True]),
 (4, [{3}, True]),
 (1, [{0, 2}, True]),
 (5, [{2, 8}, True]),
 (2, [{1, 5}, True]),
 (6, [{3}, True]),
 (7, [{3, 8}, True]),
 (3, [{4, 6, 7}, True])]

In [26]:
nodes = btc.keys()

In [27]:
nodes.take(5)

[0, 8, 4, 1, 5]

In [28]:
v_min = btc.map(lambda x: (x[0], min(x[1][0] | {x[0]})))
v_min.collect()

[(0, 0), (8, 5), (4, 3), (1, 0), (5, 2), (2, 1), (6, 3), (7, 3), (3, 3)]

In [29]:
#v_min.lookup(98)

In [30]:
NN_G_u = btc.map(lambda x: (x[0], (x[1][0] | {x[0]})))
NN_G_u.collect()

[(0, {0, 1}),
 (8, {5, 7, 8}),
 (4, {3, 4}),
 (1, {0, 1, 2}),
 (5, {2, 5, 8}),
 (2, {1, 2, 5}),
 (6, {3, 6}),
 (7, {3, 7, 8}),
 (3, {3, 4, 6, 7})]

In [31]:
addEdge1 = v_min.cogroup(NN_G_u).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
addEdge1.collect()

[(0, ([0], [{0, 1}])),
 (8, ([5], [{5, 7, 8}])),
 (1, ([0], [{0, 1, 2}])),
 (2, ([1], [{1, 2, 5}])),
 (3, ([3], [{3, 4, 6, 7}])),
 (4, ([3], [{3, 4}])),
 (5, ([2], [{2, 5, 8}])),
 (6, ([3], [{3, 6}])),
 (7, ([3], [{3, 7, 8}]))]

In [32]:
H = addEdge1.flatMap(lambda x: [(x[1][0][0], y) for y in x[1][1][0]]).map(lambda x: (x[1], x[0])).groupByKey().map(lambda x: (x[0], set(x[1]))).filter(lambda x: len(x[1]) > 1)
H.collect() #H is filtered in terms of the length

[(8, {2, 3, 5}), (1, {0, 1}), (2, {0, 1, 2}), (5, {1, 2, 5}), (7, {3, 5})]

In [33]:
v_min = H.map(lambda x: (x[0], min(x[1])))
v_min.collect()

[(8, 2), (1, 0), (2, 0), (5, 1), (7, 3)]

In [36]:
NN_H_u = H.map(lambda x: (x[0], x[1] - {min(x[1])} ))
NN_H_u.collect()

[(8, {3, 5}), (1, {1}), (2, {1, 2}), (5, {2, 5}), (7, {5})]

In [37]:
addEdge2 = v_min.cogroup(NN_H_u).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
addEdge2.collect()

[(1, ([0], [{1}])),
 (2, ([0], [{1, 2}])),
 (5, ([1], [{2, 5}])),
 (7, ([3], [{5}])),
 (8, ([2], [{3, 5}]))]

In [39]:
G = addEdge2.flatMap(lambda x: [(x[1][0][0], y) for y in x[1][1][0]]).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
G.collect()

[(0, {1, 2}), (1, {0, 2, 5}), (2, {0, 1, 3, 5}), (3, {2, 5}), (5, {1, 2, 3})]